# Making Your Model Learn Addition!
## Task 1: Introduction

Given the string "54+7", the model should return a prediction: "61".

In [1]:
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Dense, Dropout, SimpleRNN, RepeatVector
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback

from termcolor import colored

## Task 2: Generate Data
___
Note: If you are starting the notebook from this task, you can run cells from all previous tasks in the kernel by going to the top menu and then selecting Kernel > Restart and Run All
___

In [2]:
all_chars = '0123456789+'

In [4]:
num_feature = len(all_chars)
char_to_ind = {c:i for i,c in enumerate(all_chars)}
ind_to_char = {i:c for i,c in enumerate(all_chars)}

In [5]:
def generate_data():
    first = np.random.randint(0,100)
    second = np.random.randint(0,100)
    exmp = f'{first}+{second}'
    label = str(first+second)
    return exmp, label
generate_data()

('95+28', '123')

## Task 3: Create the Model
___
Note: If you are starting the notebook from this task, you can run cells from all previous tasks in the kernel by going to the top menu and then selecting Kernel > Restart and Run All
___
Consider these two reviews:

Review 1: This movie is not terrible at all.

Review 2: This movie is pretty decent.

In [7]:
hidden_units = 128
max_timesteps = 5

model = Sequential([ #encoder layer
    SimpleRNN(hidden_units, input_shape=(None, num_feature)), #decoder: output a single vector reprsentation
    RepeatVector(max_timesteps),
    SimpleRNN(hidden_units, return_sequences=True), # we want to apply a dense layer to every timestep
    TimeDistributed(Dense(num_feature, activation='softmax'))
])

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (None, 128)               17920     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 5, 128)            0         
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 5, 128)            32896     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 5, 11)             1419      
Total params: 52,235
Trainable params: 52,235
Non-trainable params: 0
_________________________________________________________________


## Task 4: Vectorize and De-Vectorize Data
___
Note: If you are starting the notebook from this task, you can run cells from all previous tasks in the kernel by going to the top menu and then selecting Kernel > Restart and Run All
___

In [16]:
def vectorize(exmp, label):
    x = np.zeros((max_timesteps, num_feature))
    y = np.zeros((max_timesteps, num_feature))
    
    diff_x = max_timesteps - len(exmp)
    diff_y = max_timesteps - len(label)
    
    for i, c in enumerate(exmp):
        x[i+diff_x, char_to_ind[c]] = 1
    for i in range(diff_x):
        x[i, char_to_ind['0']] = 1
    for i, c in enumerate(label):
        y[i+diff_y, char_to_ind[c]] = 1
    for i in range(diff_y):
        y[i, char_to_ind['0']] = 1
    return x, y
        
e, l = generate_data()
print(e, l)
x, y = vectorize(e, l)
print(x.shape, y.shape)

18+81 99
(5, 11) (5, 11)


In [18]:
def devectorize(exmp):
    result = [ind_to_char[np.argmax(vec)] for i, vec in enumerate(exmp)]
    return ''.join(result)
devectorize(y)

'00099'

## Task 5: Create Dataset
___
Note: If you are starting the notebook from this task, you can run cells from all previous tasks in the kernel by going to the top menu and then selecting Kernel > Restart and Run All
___

In [19]:
def create_dataset(num_exmp=2000):
    x = np.zeros((num_exmp, max_timesteps, num_feature))
    y = np.zeros((num_exmp, max_timesteps, num_feature))
    for i in range(num_exmp):
        e, l = generate_data()
        e_v, l_v = vectorize(e, l)
        x[i] = e_v
        y[i] = l_v
    return x, y
x, y = create_dataset()
print(x.shape, y.shape)

(2000, 5, 11) (2000, 5, 11)


In [20]:
devectorize(x[0])

'34+61'

In [21]:
devectorize(y[0])

'00095'

## Task 6: Training the Model
___
We are going to define a custom callbacks
___

In [22]:
callbacks = [LambdaCallback(
    on_epoch_end=lambda e,l: print("{:.2f}".format(l['val_acc']), end=' _ ')
), EarlyStopping(monitor='val_loss', patience=10)]

model.fit(x, y, epochs=500, batch_size=256, validation_split=0.2, verbose=False, callbacks=callbacks)

0.54 _ 0.60 _ 0.61 _ 0.61 _ 0.60 _ 0.61 _ 0.62 _ 0.64 _ 0.65 _ 0.63 _ 0.65 _ 0.65 _ 0.65 _ 0.65 _ 0.67 _ 0.67 _ 0.68 _ 0.68 _ 0.69 _ 0.70 _ 0.70 _ 0.70 _ 0.71 _ 0.71 _ 0.70 _ 0.71 _ 0.71 _ 0.72 _ 0.72 _ 0.71 _ 0.72 _ 0.74 _ 0.74 _ 0.74 _ 0.74 _ 0.75 _ 0.75 _ 0.73 _ 0.74 _ 0.74 _ 0.74 _ 0.75 _ 0.76 _ 0.76 _ 0.76 _ 0.76 _ 0.77 _ 0.76 _ 0.77 _ 0.77 _ 0.77 _ 0.78 _ 0.77 _ 0.77 _ 0.78 _ 0.78 _ 0.79 _ 0.79 _ 0.80 _ 0.80 _ 0.80 _ 0.81 _ 0.81 _ 0.81 _ 0.81 _ 0.83 _ 0.83 _ 0.83 _ 0.83 _ 0.84 _ 0.84 _ 0.84 _ 0.85 _ 0.86 _ 0.86 _ 0.86 _ 0.86 _ 0.87 _ 0.87 _ 0.87 _ 0.87 _ 0.87 _ 0.88 _ 0.88 _ 0.88 _ 0.89 _ 0.89 _ 0.89 _ 0.90 _ 0.90 _ 0.90 _ 0.90 _ 0.91 _ 0.90 _ 0.92 _ 0.91 _ 0.91 _ 0.92 _ 0.91 _ 0.92 _ 0.92 _ 0.92 _ 0.92 _ 0.92 _ 0.92 _ 0.92 _ 0.92 _ 0.92 _ 0.92 _ 0.92 _ 0.92 _ 0.93 _ 0.93 _ 0.93 _ 0.93 _ 0.93 _ 0.93 _ 0.93 _ 0.93 _ 0.93 _ 0.93 _ 0.93 _ 0.94 _ 0.93 _ 0.93 _ 0.93 _ 0.93 _ 0.93 _ 0.93 _ 0.94 _ 0.94 _ 0.94 _ 0.94 _ 0.94 _ 0.94 _ 0.94 _ 0.94 _ 0.93 _ 0.94 _ 0.94 _ 0.94 _ 0.94 _ 0.94 _

In [40]:
x_test, y_test = create_dataset(10)
preds = model.predict(x_test)

for i, pred in enumerate(preds):
    y = devectorize(y_test[i])
    y_hat = devectorize(pred)
    col = 'green' if y == y_hat else 'red'
    out = f"Input: {devectorize(x_test[i])}  -> Out: {y} _ Pred: {y_hat}"
    print(colored(out, col))

Input: 99+30  -> Out: 00129 _ Pred: 00129
Input: 98+28  -> Out: 00126 _ Pred: 00126
Input: 73+91  -> Out: 00164 _ Pred: 00164
Input: 52+79  -> Out: 00131 _ Pred: 00131
Input: 31+10  -> Out: 00041 _ Pred: 00041
Input: 03+72  -> Out: 00075 _ Pred: 00075
Input: 29+45  -> Out: 00074 _ Pred: 00074
Input: 12+57  -> Out: 00069 _ Pred: 00069
Input: 88+74  -> Out: 00162 _ Pred: 00162
Input: 90+39  -> Out: 00129 _ Pred: 00129
